In [3]:
import time
import os
from abc import ABC, abstractmethod

In [4]:
# Cryptographic libraries
from cryptography.hazmat.primitives.asymmetric import rsa, ec, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305, AESGCM

In [5]:
# ==============================================================================
# INTERFACES (COMMON RULES)
# ==============================================================================

In [6]:
class AsymmetricSystem(ABC):
    """Base class for RSA and ECC (Student 1)"""
    @abstractmethod
    def generate_keys(self):
        pass
    
    @abstractmethod
    def get_public_key_bytes(self):
        pass

class SymmetricSystem(ABC):
    """Base class for AES and ChaCha20 (Student 2)"""
    @abstractmethod
    def encrypt(self, data: bytes, key: bytes) -> bytes:
        pass
    
    @abstractmethod
    def decrypt(self, nonce, ciphertext, tag, key) -> bytes:
        pass

In [7]:
# ==============================================================================
# STUDENT 1 WORK AREA: ASYMMETRIC (RSA vs ECC)
# ==============================================================================

In [8]:
class RSA_Implementation(AsymmetricSystem):
    def __init__(self, key_size=2048):
        self.key_size = key_size
        self.private_key = None
        self.public_key = None

    def generate_keys(self):
        # RSA key generation
        self.private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=self.key_size
        )
        self.public_key = self.private_key.public_key()
    
    def get_public_key_bytes(self):
        # Serialization to simulate network transmission
        return self.public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )

In [9]:
class ECC_Implementation(AsymmetricSystem):
    def __init__(self, curve_name='SECP256R1'):
        self.curve_name = curve_name
        self.private_key = None
        self.public_key = None

    def generate_keys(self):
        # TODO: STUDENT 1 - Implement ECC key generation
        # Hint: use ec.generate_private_key(ec.SECP256R1())
        pass

    def get_public_key_bytes(self):
        # TODO: STUDENT 1 - Implement ECC serialization
        pass

In [10]:
# ==============================================================================
# STUDENT 2 WORK AREA: SYMMETRIC (AES vs ChaCha20)
# ==============================================================================

In [11]:
class AES_GCM_Implementation(SymmetricSystem):
    def encrypt(self, data: bytes, key: bytes):
        # AES-GCM requires a nonce (typically 12 bytes)
        nonce = os.urandom(12)
        aesgcm = AESGCM(key)
        ciphertext = aesgcm.encrypt(nonce, data, None)
        return nonce, ciphertext

    def decrypt(self, nonce, ciphertext, tag, key):
        # In GCM, the tag is often appended to ciphertext, but here we handle it separately
        aesgcm = AESGCM(key)
        return aesgcm.decrypt(nonce, ciphertext, None)

In [12]:
class ChaCha20_Implementation(SymmetricSystem):
    def encrypt(self, data: bytes, key: bytes):
        # TODO: STUDENT 2 - Implement ChaCha20Poly1305
        # Hint: it is very similar to AESGCM in library usage
        pass

    def decrypt(self, nonce, ciphertext, tag, key):
        # TODO: STUDENT 2 - Implement Decrypt
        pass

In [13]:
# ==============================================================================
# STUDENT 3 & 4 WORK AREA: BENCHMARK AND HYBRID PROTOCOL
# ==============================================================================

In [14]:
def run_benchmark(algo_name, operation_func, iterations=10):
    """Generic function to measure average time"""
    start_time = time.perf_counter()
    for _ in range(iterations):
        operation_func()
    end_time = time.perf_counter()
    
    avg_time = (end_time - start_time) / iterations
    print(f"[{algo_name}] Average time: {avg_time:.6f} seconds")
    return avg_time

In [15]:
print("--- STARTING CRYPTOGRAPHY BENCHMARK ---\n")

# 1. KEY GENERATION TEST (RSA vs ECC)
rsa_sys = RSA_Implementation(key_size=2048)
run_benchmark("RSA 2048 KeyGen", rsa_sys.generate_keys, iterations=20)

# ecc_sys = ECC_Implementation()
# run_benchmark("ECC 256 KeyGen", ecc_sys.generate_keys, iterations=20)

# 2. SYMMETRIC ENCRYPTION TEST (AES vs ChaCha20)
# Simulate a 1MB payload
payload = os.urandom(1024 * 1024) 
# 256-bit symmetric key (32 bytes)
sym_key = os.urandom(32) 

aes_sys = AES_GCM_Implementation()

def test_aes_encrypt():
    aes_sys.encrypt(payload, sym_key)

run_benchmark("AES-GCM 1MB Encrypt", test_aes_encrypt, iterations=50)

# TODO: Add ChaCha20 here when Student 2 is finished


--- STARTING CRYPTOGRAPHY BENCHMARK ---

[RSA 2048 KeyGen] Average time: 0.068297 seconds
[AES-GCM 1MB Encrypt] Average time: 0.000737 seconds


0.0007371639995835722